In [1]:
import numpy as np
import pandas as pd

import edhec_risk_kit_206 as erk

%load_ext autoreload
%autoreload 2

In [3]:
# inds = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth',
#        'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry',
#        'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper',
#        'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']

# inds=['Beer', 'Hlth', 'Fin','Rtail','Whlsl']

ind_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["1974":]
ind_mcap = erk.get_ind_market_caps(49, weights=True)["1974":]

rets = ind_rets["2014":]
mcap = ind_mcap["2014":]
weights = mcap.iloc[0]
cov = rets.cov()

In [4]:
def risk_contribution(w,cov):
    """
    Compute the contributions to risk of the constituents of a portfolio, given a set of portfolio weights and a covariance matrix
    """
    total_portfolio_var = erk.portfolio_vol(w,cov)**2
    # Marginal contribution of each constituent
    marginal_contrib = cov@w
    risk_contrib = np.multiply(marginal_contrib,w.T)/total_portfolio_var
    return risk_contrib

#### q1.  & q2

In [10]:
risk_contrib = risk_contribution(weights, cov)
risk_contrib.sort_values(ascending=False).head()

Banks    0.104060
Softw    0.093745
Oil      0.088914
Drugs    0.084653
Rtail    0.070120
dtype: float64

#### q3.

In [11]:
risk_contribution(erk.weight_ew(rets),cov).sort_values(ascending=False).head()

Steel    0.030938
Ships    0.030005
Coal     0.029025
Mines    0.028641
BldMt    0.028106
dtype: float64

In [12]:
from scipy.optimize import minimize

def target_risk_contributions(target_risk, cov):
    """
    Returns the weights of the portfolio that gives you the weights such
    that the contributions to portfolio risk are as close as possible to
    the target_risk, given the covariance matrix
    """
    n = cov.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1.0),) * n # an N-tuple of 2-tuples!
    # construct the constraints
    weights_sum_to_1 = {'type': 'eq',
                        'fun': lambda weights: np.sum(weights) - 1
    }
    def msd_risk(weights, target_risk, cov):
        """
        Returns the Mean Squared Difference in risk contributions
        between weights and target_risk
        """
        w_contribs = risk_contribution(weights, cov)
        return ((w_contribs-target_risk)**2).sum()
    
    weights = minimize(msd_risk, init_guess,
                       args=(target_risk, cov), method='SLSQP',
                       options={'disp': False},
                       constraints=(weights_sum_to_1,),
                       bounds=bounds)
    return weights.x

def equal_risk_contributions(cov):
    """
    Returns the weights of the portfolio that equalizes the contributions
    of the constituents based on the given covariance matrix
    """
    n = cov.shape[0]
    return target_risk_contributions(target_risk=np.repeat(1/n,n), cov=cov)


In [16]:
erc_weights = pd.Series(equal_risk_contributions(cov), index=cov.index)
erc_weights.sort_values(ascending=False).head(3)

Util     0.052156
Hshld    0.033184
Smoke    0.032606
dtype: float64

In [19]:
erc_weights.sort_values(ascending=True).head(3)

Steel    0.012809
Ships    0.013041
BldMt    0.013790
dtype: float64

In [21]:
sorted(risk_contrib)[-1] - sorted(risk_contrib)[0]

0.1039644943981684

In [24]:
ew_rc = risk_contribution(erk.weight_ew(rets),cov)
(sorted(ew_rc)[-1]-sorted(ew_rc)[0])*100

2.5021757624911465